In [1]:
import matplotlib.pyplot as plt
import numpy as np
from keras import layers, models
from keras.preprocessing.image import load_img, img_to_array
from keras.applications import vgg19
import keras.backend as K
from scipy.optimize import fmin_l_bfgs_b
from scipy.misc import imsave

%matplotlib inline

Using TensorFlow backend.


In [3]:
# Set up initial values
content_img_path = '../data/neural_style_images/professional_pic.jpg'
style_img_path = '../data/neural_style_images/popova.jpeg'

width, height = load_img(content_img_path).size
img_height = 400
img_width = (width * img_height) // height

In [4]:
width, height

(396, 396)

In [5]:
img_height, img_width

(400, 400)

In [6]:
def preprocess_image(image_path, img_height, img_width):
    
    img = load_img(image_path, target_size=(img_height, img_width))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    
    return img


def deprocess_image(img):
    img[:, :, 0] += 103.99
    img[:, :, 1] += 116.779
    img[:, :, 2] += 123.68
    img = img[:, :, ::-1]
    
    return img

In [11]:
# Content image and style image are static
content_img = K.constant(preprocess_image(content_img_path, img_height, img_width))
style_img = K.constant(preprocess_image(style_img_path, img_height, img_width))

# Generated image will take numpy arrays
generated_img = K.placeholder(shape=(1, img_height, img_width, 3))

# Input of model will be three images
input_tensor = K.concatenate([content_img, style_img, generated_img], axis=0)

# Will use the convolution base of VGG19 pretrained on imagenet
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights="imagenet",
                    include_top=False)

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [ ]:
def content_loss(target, generated):
    loss = K.sum(K.square(generated - target))
    
    return loss


def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, [2, 0, 1]))
    gram = K.dot(features, K.transpose(features))
    
    return gram


def style_loss(style_img, generated_img):
    S = gram_matrix(style_img)
    G = gram_matrix(generated_img)
    loss = (1 / ((2 * img_height * img_width * 3) ** 2)) * K.sum(K.square(S - G))
    
    return loss

In [15]:
output_dict = dict([(layer.name, layer.output) for layer in model.layers])
content_layer = "block5_conv1"
style_layers = ["block1_conv1", "block2_conv1", "block3_conv1", "block4_conv1", "block5_conv1"]
style_weight = 1
content_weight = 0.025

loss = K.variable(0.)
feature_layers = output_dict[content_layer]
content_img_features = feature_layers[0, :, : , :]
generated_img_features = feature_layers[2, :, :, :]
loss += content_loss(content_img_features, generated_img_features)

for layer in style_layers:
    layer_features = output_dict[layer]
    style_img_features = layer_features[1, :, :, :]
    generated_img_features = layer_features[2, :, :, :]
    loss += (style_weight / len(style_layers)) * style_loss(style_img_features, generated_img_features)


grads = K.gradients(loss, generated_img)[0]
fetch_loss_and_gradients = K.function([generated_img], [loss, grads])


class Evaluator:
    def __init__(self):
        self.loss_value = None
        self.grads_values = None
        
    def loss(self, x):
        assert self.loss_value is None
        x = x.reshape((1, img_height, img_width, 3))
        outs = fetch_loss_and_gradients([x])
        loss_value = outs[0]
        grads_values = outs[1].flatten().astype("float32")
        self.loss_value = loss_value
        self.grads_values = grads_values
        
        return self.loss_value
    
    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grads_values)
        self.loss_value = None
        self.grads_values = None
        
        return grad_values

In [17]:
evaluator = Evaluator()
x = preprocess_image(content_img_path)
x = x.flatten()

for i in range(20):
    print(f"Start of iteration {i}.")
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x, fprime=evaluator.grads, maxfun=20)
    img = x.copy().reshape((img_height, img_width, 3))
    img = deprocess_image(img)
    fname = f"result_at_iteration_{i}.png"
    imsave(fname, img)

Start of iteration 0.


ValueError: failed to initialize intent(inout) array -- expected elsize=8 but got 4

In [19]:
import time
result_prefix = 'style_transfer_result'
iterations = 20

# Run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss.
# This is our initial state: the target image.
# Note that `scipy.optimize.fmin_l_bfgs_b` can only process flat vectors.
x = preprocess_image(content_img_path)
x = x.flatten()
for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x,
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    # Save current generated image
    img = x.copy().reshape((img_height, img_width, 3))
    img = deprocess_image(img)
    fname = result_prefix + '_at_iteration_%d.png' % i
    imsave(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration 0


ValueError: failed to initialize intent(inout) array -- expected elsize=8 but got 4

In [20]:

from keras.preprocessing.image import load_img, img_to_array

# This is the path to the image you want to transform.
target_image_path = 'professional_pic.jpg'
# This is the path to the style image.
style_reference_image_path = 'popova.jpeg'

# Dimensions of the generated picture.
width, height = load_img(target_image_path).size
img_height = 400
img_width = int(width * img_height / height)

def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_height, img_width))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

def deprocess_image(x):
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x


from keras import backend as K

target_image = K.constant(preprocess_image(target_image_path))
style_reference_image = K.constant(preprocess_image(style_reference_image_path))

# This placeholder will contain our generated image
combination_image = K.placeholder((1, img_height, img_width, 3))

# We combine the 3 images into a single batch
input_tensor = K.concatenate([target_image,
                              style_reference_image,
                              combination_image], axis=0)

# We build the VGG19 network with our batch of 3 images as input.
# The model will be loaded with pre-trained ImageNet weights.
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet',
                    include_top=False)
print('Model loaded.')


def content_loss(base, combination):
    return K.sum(K.square(combination - base))


def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram


def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_height * img_width
    return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))


def total_variation_loss(x):
    a = K.square(
        x[:, :img_height - 1, :img_width - 1, :] - x[:, 1:, :img_width - 1, :])
    b = K.square(
        x[:, :img_height - 1, :img_width - 1, :] - x[:, :img_height - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))


# Dict mapping layer names to activation tensors
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
# Name of layer used for content loss
content_layer = 'block5_conv2'
# Name of layers used for style loss
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
                'block5_conv1']
# Weights in the weighted average of the loss components
total_variation_weight = 1e-4
style_weight = 1.
content_weight = 0.025

# Define the loss by adding all components to a `loss` variable
loss = K.variable(0.)
layer_features = outputs_dict[content_layer]
target_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss += content_weight * content_loss(target_image_features,
                                      combination_features)
for layer_name in style_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss += (style_weight / len(style_layers)) * sl
loss += total_variation_weight * total_variation_loss(combination_image)


# Get the gradients of the generated image wrt the loss
grads = K.gradients(loss, combination_image)[0]

# Function to fetch the values of the current loss and the current gradients
fetch_loss_and_grads = K.function([combination_image], [loss, grads])


class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        x = x.reshape((1, img_height, img_width, 3))
        outs = fetch_loss_and_grads([x])
        loss_value = outs[0]
        grad_values = outs[1].flatten().astype('float64')
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

evaluator = Evaluator()


from scipy.optimize import fmin_l_bfgs_b
from scipy.misc import imsave
import time

result_prefix = 'style_transfer_result'
iterations = 20

# Run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss.
# This is our initial state: the target image.
# Note that `scipy.optimize.fmin_l_bfgs_b` can only process flat vectors.
x = preprocess_image(target_image_path)
x = x.flatten()
for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x,
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    # Save current generated image
    img = x.copy().reshape((img_height, img_width, 3))
    img = deprocess_image(img)
    fname = result_prefix + '_at_iteration_%d.png' % i
    imsave(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Model loaded.
Start of iteration 0
Current loss value: 8.07968e+09
Image saved as style_transfer_result_at_iteration_0.png
Iteration 0 completed in 217s
Start of iteration 1


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:166: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


Current loss value: 3.08114e+09
Image saved as style_transfer_result_at_iteration_1.png
Iteration 1 completed in 211s
Start of iteration 2
Current loss value: 1.98132e+09
Image saved as style_transfer_result_at_iteration_2.png
Iteration 2 completed in 221s
Start of iteration 3
Current loss value: 1.38261e+09
Image saved as style_transfer_result_at_iteration_3.png
Iteration 3 completed in 219s
Start of iteration 4
Current loss value: 1.10522e+09
Image saved as style_transfer_result_at_iteration_4.png
Iteration 4 completed in 206s
Start of iteration 5
Current loss value: 9.25592e+08
Image saved as style_transfer_result_at_iteration_5.png
Iteration 5 completed in 208s
Start of iteration 6
Current loss value: 8.11377e+08
Image saved as style_transfer_result_at_iteration_6.png
Iteration 6 completed in 211s
Start of iteration 7
Current loss value: 7.3148e+08
Image saved as style_transfer_result_at_iteration_7.png
Iteration 7 completed in 209s
Start of iteration 8
Current loss value: 6.64767e